<a href="https://colab.research.google.com/github/nb20593/data-science/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing transformers

In [1]:
pip install transformers

     |████████████████████████████████| 2.1MB 6.1MB/s 
     |████████████████████████████████| 3.3MB 30.1MB/s 
     |████████████████████████████████| 901kB 43.9MB/s 


import packages

In [2]:
import numpy as np
import pandas as pd
import string
import re
from matplotlib import pyplot as plt
import torch
from sklearn.externals import joblib
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


datasets

In [3]:
irony_train_label_data =  pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_labels.txt',header=None)
irony_train_text_data = pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/train_text.txt',header=None)
irony_test_label_data =pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_labels.txt',header=None)
irony_test_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/test_text.txt',header=None)
irony_val_label_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_labels.txt',header=None)
irony_val_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/irony/val_text.txt',header=None)

Verify the shapes

In [4]:
print(irony_train_label_data.shape)
print(irony_train_text_data.shape)
print(irony_test_label_data.shape)
print(irony_test_text_data.shape)
print(irony_val_label_data.shape)
print(irony_val_text_data.shape)

(2862, 1)
(2862, 1)
(784, 1)
(784, 1)
(955, 1)
(955, 1)


Combine text and label columns


In [5]:
df_irony_train = irony_train_text_data.join(irony_train_label_data,rsuffix = "_2")
df_irony_test = irony_test_text_data.join(irony_test_label_data,rsuffix = "_2")
df_irony_val = irony_val_text_data.join(irony_val_label_data,rsuffix = "_2")
df_irony_train.columns = ['text','label']
df_irony_test.columns = ['text','label']
df_irony_val.columns = ['text','label']
df_irony_val.head(5)

,text,label
0,#NBA players #NY support protests of #police k...,1
1,A new year about to start|So many people came ...,0
2,"Obama's $1,176,120.90 in Taxpayer Funded Costs...",1
3,Can't wait to work with the dream team again t...,1
4,!!! RT @user Of all the places to get stuck in...,1


Remove contradictions

In [6]:
 contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

df_irony_train['text']=df_irony_train['text'].apply(lambda x:expand_contractions(x))
df_irony_test['text'] = df_irony_test['text'].apply(lambda x:expand_contractions(x))
df_irony_val['text'] = df_irony_val['text'].apply(lambda x:expand_contractions(x))

Preprocessing

In [7]:
def processTweet(tweet):
    # Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','',tweet)
    # Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    # To lowercase
    tweet = tweet.lower()
    # Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w*', '', tweet)
    # Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    # Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove digits
    tweet = re.sub(r'\w*\d\w*','', tweet)
    #Remove puntuation
    tweet = re.sub(r'[%s]' % re.escape(string.punctuation), '', tweet)
    #Remove extra spaces
    tweet = re.sub(' +',' ',tweet)
    return tweet

In [8]:
df_irony_train['text'] = df_irony_train['text'].apply(processTweet)
df_irony_test['text'] = df_irony_test['text'].apply(processTweet)
df_irony_val['text'] = df_irony_val['text'].apply(processTweet)

Lemmatization

In [ ]:
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

df_irony_train['text']=df_irony_train['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_irony_test['text']=df_irony_test['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_irony_val['text']=df_irony_val['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
train_tokenized = df_irony_train['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
test_tokenized = df_irony_test['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
val_tokenized = df_irony_val['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))

In [ ]:
max_len = 0
for i in train_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

train_padded = np.array([i + [0]*(40-len(i)) for i in train_tokenized.values])
test_padded = np.array([i + [0]*(40-len(i)) for i in test_tokenized.values])
val_padded = np.array([i + [0]*(40-len(i)) for i in val_tokenized.values])

In [ ]:
print(np.array(test_padded).shape,np.array(train_padded).shape,np.array(val_padded).shape)

In [ ]:
train_attention_mask = np.where(train_padded != 0, 1, 0)
test_attention_mask = np.where(test_padded != 0, 1, 0)
val_attention_mask = np.where(val_padded != 0,1,0)
test_attention_mask.shape

In [ ]:
train_input_ids = torch.tensor(train_padded)  
train_attention_mask = torch.tensor(train_attention_mask)

with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

In [ ]:
test_input_ids = torch.tensor(test_padded)
test_attention_mask = torch.tensor(test_attention_mask)

with torch.no_grad():
   test_last_hidden_states = model(test_input_ids, attention_mask=test_attention_mask)

In [ ]:
val_input_ids = torch.tensor(val_padded)
val_attention_mask = torch.tensor(val_attention_mask)

with torch.no_grad():
   val_last_hidden_states = model(val_input_ids, attention_mask=val_attention_mask)

In [ ]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
test_features = test_last_hidden_states[0][:,0,:].numpy()
val_features = val_last_hidden_states[0][:,0,:].numpy()
train_labels = torch.tensor(df_irony_train['label'])
test_labels = torch.tensor(df_irony_test['label'])
val_labels = torch.tensor(df_irony_val['label'])

In [ ]:
random_forest.fit(train_features, train_labels)
random_forest.score(test_features, test_labels)

In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
lr_clf.score(val_features, val_labels)

In [ ]:
from sklearn.svm import SVC
svc_regressor = SVC()

scores = cross_val_score(svc_regressor, train_features, train_labels)
print("SVM classifier average score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(train_features,train_labels)
print(grid.best_estimator_)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 
grid_predictions = grid.predict(test_features)
print(confusion_matrix(test_labels,grid_predictions))
print(classification_report(test_labels,grid_predictions))
grid_predictions = grid.predict(val_features)
print(classification_report(val_labels,grid_predictions))

In [ ]:
svc_regressor.fit(train_features, train_labels)
svc_regressor.score(test_features, test_labels)
print ("accuracy from svm: ",svc_regressor.score(test_features, test_labels))

In [ ]:

from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(2000), learning_rate='adaptive', verbose=False)
neural_network.fit(train_features, train_labels)
neural_network.score(test_features, test_labels)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, train_features, train_labels)
print("Random forest classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
title = r"Learning Curves (SVM, sigmoid kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=10)
estimator = SVC(gamma=0.001,kernel = 'sigmoid')
plot_learning_curve(estimator, title,train_features, train_labels,cv=cv, n_jobs=4)
plt.show()

In [ ]:
offensive_train_label_data =  pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt',header=None)
offensive_train_text_data = pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt',header=None)
offensive_test_label_data =pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt',header=None)
offensive_test_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt',header=None)
offensive_val_label_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt',header=None)
offensive_val_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt',header=None)

Verify the shapes

In [ ]:
offensive_train_text_data = offensive_train_text_data.drop(offensive_train_text_data.columns[1:], axis = 1)
offensive_test_text_data = offensive_train_text_data.drop(offensive_train_text_data.columns[1:], axis = 1)
offensive_val_text_data = offensive_train_text_data.drop(offensive_train_text_data.columns[1:], axis = 1)
print(offensive_train_label_data.shape)
print(offensive_train_text_data.shape)
print(offensive_test_label_data.shape)
print(offensive_test_text_data.shape)
print(offensive_val_label_data.shape)
print(offensive_val_text_data.shape)

(11916, 1)
(11916, 1)
(860, 1)
(11916, 1)
(1324, 1)
(11916, 1)


Combine text and label columns


In [ ]:
df_offensive_train = offensive_train_text_data.join(offensive_train_label_data,rsuffix = "_2")
df_offensive_test = offensive_test_text_data.join(offensive_test_label_data,rsuffix = "_2")
df_offensive_val = offensive_val_text_data.join(offensive_val_label_data,rsuffix = "_2")
df_offensive_train.columns = ['text','label']
df_offensive_test.columns = ['text','label']
df_offensive_val.columns = ['text','label']
df_offensive_val.head(5)

,text,label
0,@user Bono... who cares. Soon people will unde...,0.0
1,@user Eight years the republicans denied obama...,1.0
2,@user Get him some line help. He is gonna be j...,0.0
3,@user @user She is great. Hi Fiona!,0.0
4,@user She has become a parody unto herself? Sh...,0.0


Remove contradictions

In [ ]:
 contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

df_offensive_train['text']=df_offensive_train['text'].apply(lambda x:expand_contractions(x))
df_offensive_test['text'] = df_offensive_test['text'].apply(lambda x:expand_contractions(x))
df_offensive_val['text'] = df_offensive_val['text'].apply(lambda x:expand_contractions(x))

Preprocessing

In [ ]:
df_offensive_train['text'] = df_offensive_train['text'].apply(processTweet)
df_offensive_test['text'] = df_offensive_test['text'].apply(processTweet)
df_offensive_val['text'] = df_offensive_val['text'].apply(processTweet)

Lemmatization

In [ ]:
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

df_offensive_train['text']=df_offensive_train['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_offensive_test['text']=df_offensive_test['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_offensive_val['text']=df_offensive_val['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
train_tokenized = df_offensive_train['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
test_tokenized = df_offensive_test['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
val_tokenized = df_offensive_val['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))

In [ ]:
max_len = 0
for i in train_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

train_padded = np.array([i + [0]*(max_len-len(i)) for i in train_tokenized.values])
test_padded = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized.values])
val_padded = np.array([i + [0]*(max_len-len(i)) for i in val_tokenized.values])

In [ ]:
print(np.array(test_padded).shape,np.array(train_padded).shape,np.array(val_padded).shape)

(11916, 65) (11916, 65) (11916, 65)


In [ ]:
train_attention_mask = np.where(train_padded != 0, 1, 0)
test_attention_mask = np.where(test_padded != 0, 1, 0)
val_attention_mask = np.where(val_padded != 0,1,0)
test_attention_mask.shape

(11916, 65)

In [ ]:
train_input_ids = torch.tensor(train_padded)  
train_attention_mask = torch.tensor(train_attention_mask)

with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

NameError: ignored

In [ ]:
test_input_ids = torch.tensor(test_padded)
test_attention_mask = torch.tensor(test_attention_mask)

with torch.no_grad():
   test_last_hidden_states = model(test_input_ids, attention_mask=test_attention_mask)

In [ ]:
val_input_ids = torch.tensor(val_padded)
val_attention_mask = torch.tensor(val_attention_mask)

with torch.no_grad():
   val_last_hidden_states = model(val_input_ids, attention_mask=val_attention_mask)

In [ ]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
test_features = test_last_hidden_states[0][:,0,:].numpy()
val_features = val_last_hidden_states[0][:,0,:].numpy()
train_labels = torch.tensor(df_offensive_train['label'])
test_labels = torch.tensor(df_offensive_test['label'])
val_labels = torch.tensor(df_offensive_val['label'])

In [ ]:
random_forest.fit(train_features, train_labels)
random_forest.score(test_features, test_labels)

In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
lr_clf.score(val_features, val_labels)

In [ ]:
from sklearn.svm import SVC
svc_regressor = SVC()

scores = cross_val_score(svc_regressor, train_features, train_labels)
print("SVM classifier average score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(train_features,train_labels)
print(grid.best_estimator_)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 
grid_predictions = grid.predict(test_features)
print(confusion_matrix(test_labels,grid_predictions))
print(classification_report(test_labels,grid_predictions))
grid_predictions = grid.predict(val_features)
print(classification_report(val_labels,grid_predictions))

In [ ]:
svc_regressor.fit(train_features, train_labels)
svc_regressor.score(test_features, test_labels)
print ("accuracy from svm: ",svc_regressor.score(test_features, test_labels))

In [ ]:

from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(2000), learning_rate='adaptive', verbose=False)
neural_network.fit(train_features, train_labels)
neural_network.score(test_features, test_labels)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, train_features, train_labels)
print("Random forest classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
title = r"Learning Curves (SVM, sigmoid kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=15)
estimator = SVC(gamma=0.001,kernel = 'sigmoid')
plot_learning_curve(estimator, title,train_features, train_labels,cv=cv, n_jobs=4)
plt.show()

datasets

In [ ]:
sentiment_train_label_data =  pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt',header=None)
sentiment_train_text_data = pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt',header=None)
sentiment_test_label_data =pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt',header=None)
sentiment_test_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt',header=None)
sentiment_val_label_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_labels.txt',header=None)
sentiment_val_text_data = pd.read_fwf ('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/val_text.txt',header=None)

Verify the shapes

In [ ]:
sentiment_train_text_data = sentiment_train_text_data.drop(sentiment_train_text_data.columns[1], axis = 1)
sentiment_val_text_data = sentiment_val_text_data.drop(sentiment_val_text_data.columns[1], axis = 1)
print(sentiment_train_label_data.shape)
print(sentiment_train_text_data.shape)
print(sentiment_test_label_data.shape)
print(sentiment_test_text_data.shape)
print(sentiment_val_label_data.shape)
print(sentiment_val_text_data.shape)
sentiment_val_text_data.head(20)

(45615, 1)
(45615, 1)
(12284, 1)
(12284, 1)
(2000, 1)
(2000, 1)


,0
0,Dark Souls 3 April Launch Date Confirmed With ...
1,"""National hot dog day, national tequila day, t..."
2,When girls become bandwagon fans of the Packer...
3,@user I may or may not have searched it up on ...
4,Here's your starting TUESDAY MORNING Line up a...
5,"@user F-Main, are you in the office tomorrow i..."
6,#US 1st Lady Michelle Obama speaking at the 20...
7,Omg this show is so predictable even for the 3...
8,"""What a round by Paul Dunne, good luck tomorro..."
9,Irving Plaza NYC Blackout Saturday night. Got ...


Combine text and label columns


In [ ]:
df_sentiment_train = sentiment_train_text_data.join(sentiment_train_label_data,rsuffix = "_2")
df_sentiment_test = sentiment_test_text_data.join(sentiment_test_label_data,rsuffix = "_2")
df_sentiment_val = sentiment_val_text_data.join(sentiment_val_label_data,rsuffix = "_2")
df_sentiment_train.columns = ['text','label']
df_sentiment_test.columns = ['text','label']
df_sentiment_val.columns = ['text','label']
df_sentiment_val.head(5)

,text,label
0,Dark Souls 3 April Launch Date Confirmed With ...,1
1,"""National hot dog day, national tequila day, t...",2
2,When girls become bandwagon fans of the Packer...,0
3,@user I may or may not have searched it up on ...,1
4,Here's your starting TUESDAY MORNING Line up a...,1


Remove contradictions

In [ ]:
 contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

df_sentiment_train['text']=df_sentiment_train['text'].apply(lambda x:expand_contractions(x))
df_sentiment_test['text'] = df_sentiment_test['text'].apply(lambda x:expand_contractions(x))
df_sentiment_val['text'] = df_sentiment_val['text'].apply(lambda x:expand_contractions(x))

Preprocessing

In [ ]:
df_sentiment_train['text'] = df_sentiment_train['text'].apply(processTweet)
df_sentiment_test['text'] = df_sentiment_test['text'].apply(processTweet)
df_sentiment_val['text'] = df_sentiment_val['text'].apply(processTweet)

Lemmatization

In [ ]:
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

df_sentiment_train['text']=df_sentiment_train['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_sentiment_test['text']=df_sentiment_test['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
df_sentiment_val['text']=df_sentiment_val['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
train_tokenized = df_sentiment_train['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
test_tokenized = df_sentiment_test['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))
val_tokenized = df_sentiment_val['text'].apply((lambda x: tokenizer.encode(x,add_special_tokens=True,padding=True,truncation=True)))

In [ ]:
max_len = 0
for i in train_tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

train_padded = np.array([i + [0]*(70-len(i)) for i in train_tokenized.values])
test_padded = np.array([i + [0]*(70-len(i)) for i in test_tokenized.values])
val_padded = np.array([i + [0]*(70-len(i)) for i in val_tokenized.values])

In [ ]:
print(np.array(test_padded).shape,np.array(train_padded).shape,np.array(val_padded).shape)

(12284, 70) (45615, 70) (2000, 70)


In [ ]:
train_attention_mask = np.where(train_padded != 0, 1, 0)
test_attention_mask = np.where(test_padded != 0, 1, 0)
val_attention_mask = np.where(val_padded != 0,1,0)
test_attention_mask.shape

(12284, 70)

In [ ]:
train_input_ids = torch.tensor(train_padded)  
train_attention_mask = torch.tensor(train_attention_mask)

with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

In [ ]:
test_input_ids = torch.tensor(test_padded)
test_attention_mask = torch.tensor(test_attention_mask)

with torch.no_grad():
   test_last_hidden_states = model(test_input_ids, attention_mask=test_attention_mask)

In [ ]:
val_input_ids = torch.tensor(val_padded)
val_attention_mask = torch.tensor(val_attention_mask)

with torch.no_grad():
   val_last_hidden_states = model(val_input_ids, attention_mask=val_attention_mask)

In [ ]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
test_features = test_last_hidden_states[0][:,0,:].numpy()
val_features = val_last_hidden_states[0][:,0,:].numpy()
train_labels = torch.tensor(df_sentiment_train['label'])
test_labels = torch.tensor(df_sentiment_test['label'])
val_labels = torch.tensor(df_sentiment_val['label'])

In [ ]:
random_forest.fit(train_features, train_labels)
random_forest.score(test_features, test_labels)

In [ ]:
lr_clf = LogisticRegression(max_iter=1000)
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)
lr_clf.score(val_features, val_labels)

In [ ]:
from sklearn.svm import SVC
svc_regressor = SVC()

scores = cross_val_score(svc_regressor, train_features, train_labels)
print("SVM classifier average score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(train_features,train_labels)
print(grid.best_estimator_)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix 
grid_predictions = grid.predict(test_features)
print(confusion_matrix(test_labels,grid_predictions))
print(classification_report(test_labels,grid_predictions))

In [ ]:
svc_regressor.fit(train_features, train_labels)
svc_regressor.score(test_features, test_labels)
print ("accuracy from svm: ",svc_regressor.score(val_features, val_labels))

In [ ]:

from sklearn.neural_network import MLPClassifier
neural_network = MLPClassifier(hidden_layer_sizes=(2000), learning_rate='adaptive', verbose=False)
neural_network.fit(train_features, train_labels)
neural_network.score(test_features, test_labels)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, train_features, train_labels)
print("Random forest classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
title = r"Learning Curves (SVM, sigmoid kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001,kernel = 'sigmoid')
plot_learning_curve(estimator, title,train_features, train_labels,cv=cv, n_jobs=4)
plt.show()